In [1]:
from datetime import datetime as dt

from src.shared import Guild, Channel, ChannelType, Message, User, login_utils
from src.server import db, asset_generator

In [2]:
# dangerous!!!
db.guilds.drop()
db.channels.drop()
db.messages.drop()
db.users.drop()
db.passwords.drop()

In [3]:
db.messages.create_index("channel_id")
db.users.create_index("username", unique=True)
db.channels.create_index("guild_id")
db.guilds.create_index("join_code", unique=True)

'join_code_1'

In [4]:
u = asset_generator.generate_user("test", login_utils.hash_password("Pass1234"))

In [5]:
guilds = [asset_generator.generate_guild(f"Test guild {i}", u.id) for i in range(5)]


In [6]:
u2 = asset_generator.generate_user("amogus", login_utils.hash_password("Amogus123"))

In [7]:
for guild in guilds:
    db.user_join_guild(u2.id, guild.id)

In [8]:
for guild in guilds:
    for i in range(5):
        asset_generator.generate_channel(f"test-channel-{i}", ChannelType.TEXT, guild.id)

In [9]:
for channel_db in db.channels.find():
    channel: Channel = Channel.from_db_dict(channel_db)
    buffer: list[dict] = []
    for i in range(150):
        id = asset_generator.get_id()
        buffer.insert(0, Message(id, channel.id, f"test message {i} in channel {channel.name} in guild {channel.guild_id}", attachment_id=0, author=u, timestamp=id >> asset_generator.TAG_LENGTH).to_db_dict())
    db.messages.insert_many(buffer)


In [10]:
db.get_messages(channel.id, 0, 10)

[Message(id=7192203218652869, channel_id=7192203218649119, content='test message 149 in channel test-channel-4 in guild 7192203214454789', attachment_id=0, author=User(id=7192203214454784, username='test'), timestamp=1714754872),
 Message(id=7192203218652868, channel_id=7192203218649119, content='test message 148 in channel test-channel-4 in guild 7192203214454789', attachment_id=0, author=User(id=7192203214454784, username='test'), timestamp=1714754872),
 Message(id=7192203218652867, channel_id=7192203218649119, content='test message 147 in channel test-channel-4 in guild 7192203214454789', attachment_id=0, author=User(id=7192203214454784, username='test'), timestamp=1714754872),
 Message(id=7192203218652866, channel_id=7192203218649119, content='test message 146 in channel test-channel-4 in guild 7192203214454789', attachment_id=0, author=User(id=7192203214454784, username='test'), timestamp=1714754872),
 Message(id=7192203218652865, channel_id=7192203218649119, content='test message